## Install dependencies

In [ ]:
!git clone https://github.com/ZurichNLP/xstance.git

Cloning into 'xstance'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 80 (delta 36), reused 56 (delta 17), pack-reused 0
Unpacking objects: 100% (80/80), 6.94 MiB | 9.42 MiB/s, done.


In [ ]:
!unzip xstance/data/xstance-data-v1.0.zip -d xstance/data

Archive:  xstance/data/xstance-data-v1.0.zip
  inflating: xstance/data/valid.jsonl  
  inflating: xstance/data/train.jsonl  
  inflating: xstance/data/test.jsonl  
  inflating: xstance/data/questions.it.jsonl  
  inflating: xstance/data/questions.fr.jsonl  
  inflating: xstance/data/questions.en.jsonl  
  inflating: xstance/data/questions.de.jsonl  
  inflating: xstance/data/LICENSE    


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import shutil
from pathlib import Path

import evaluate
import jsonlines
import numpy as np
import transformers
from datasets import load_dataset, disable_caching
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

## Load model and pre-process data

In [ ]:
model_name = "ZurichNLP/swissbert"

In [ ]:
model_dir = Path("finetuned_models") / model_name
model_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
dataset = load_dataset("x_stance")

Generating train split:   0%|          | 0/45640 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/17705 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3926 [00:00<?, ? examples/s]

Dataset x_stance downloaded and prepared to /root/.cache/huggingface/datasets/x_stance/default/0.1.0/a5dbbc65742772e61dfd52276a9de65f4789c429bf30d549e983c8c65b72347c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Print an example
dataset["validation"][0]

{'question': 'Sollen Ausländer/-innen, die seit mindestens zehn Jahren in der Schweiz leben, das Stimm- und Wahlrecht auf Gemeindeebene erhalten?',
 'id': 17188,
 'question_id': 3427,
 'language': 'de',
 'comment': 'Ich bin finde das geht zu wenig weit. Alle Menschen die hier leben sollen das Recht auf Mitsprache haben.',
 'label': 'FAVOR',
 'numerical_label': 100,
 'author': '89900aa69be7',
 'topic': 'Immigration'}

In [ ]:
# Enumerate all possible labels
set(dataset["validation"]["label"])

{'AGAINST', 'FAVOR'}

In [ ]:
id2label = {0: "AGAINST", 1: "FAVOR"}
label2id = {"AGAINST": 0, "FAVOR": 1}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
adapter_map = {
    "de": 0,
    "fr": 1,
    "it": 2,
}

def preprocess(examples):
    examples["label"] = [label2id[label] for label in examples["label"]]
    examples["lang_ids"] = [adapter_map[language] for language in examples["language"]]
    examples = tokenizer(
        examples["question"],
        examples["comment"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    return examples

In [ ]:
preprocessed_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/45640 [00:00<?, ? examples/s]

Map:   0%|          | 0/17705 [00:00<?, ? examples/s]

Map:   0%|          | 0/3926 [00:00<?, ? examples/s]

In [ ]:
# Print a pre-processed example
preprocessed_dataset["validation"][0]

{'question': 'Sollen Ausländer/-innen, die seit mindestens zehn Jahren in der Schweiz leben, das Stimm- und Wahlrecht auf Gemeindeebene erhalten?',
 'id': 17188,
 'question_id': 3427,
 'language': 'de',
 'comment': 'Ich bin finde das geht zu wenig weit. Alle Menschen die hier leben sollen das Recht auf Mitsprache haben.',
 'label': 1,
 'numerical_label': 100,
 'author': '89900aa69be7',
 'topic': 'Immigration',
 'lang_ids': 0,
 'input_ids': [0,
  11899,
  21,
  4332,
  132,
  14,
  624,
  9,
  13,
  250,
  1724,
  835,
  203,
  20,
  12,
  214,
  2664,
  9,
  41,
  20752,
  14,
  18,
  883,
  2249,
  39,
  319,
  16498,
  922,
  103,
  2,
  2,
  591,
  854,
  6100,
  41,
  391,
  38,
  847,
  1158,
  8,
  1675,
  418,
  13,
  323,
  2664,
  494,
  41,
  2164,
  39,
  208,
  8083,
  127,
  8,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [ ]:
train_dataset = preprocessed_dataset["train"].shuffle(seed=42)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at ZurichNLP/swissbert were not used when initializing XmodForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XmodForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XmodForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XmodForSequenceClassification were not initialized from the model checkpoint at ZurichNLP/swissbert and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should 

In [ ]:
# Freeze language adapters
for layer in model.roberta.encoder.layer:
    if layer.output.adapter_layer_norm is not None:
        for parameter in layer.output.adapter_layer_norm.parameters():
            parameter.requires_grad = False
    for parameter in layer.output.adapter_modules.parameters():
        parameter.requires_grad = False

## Fine-tune the model

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Set to 3 to reproduce results in paper
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=553589,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=preprocessed_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.504100,0.500317,0.755476


TrainOutput(global_step=2853, training_loss=0.5488554014893862, metrics={'train_runtime': 2188.6097, 'train_samples_per_second': 20.853, 'train_steps_per_second': 1.304, 'total_flos': 7992796563333120.0, 'train_loss': 0.5488554014893862, 'epoch': 1.0})

## Evaluate on the test sets

In [ ]:
output = trainer.predict(preprocessed_dataset["test"])

In [ ]:
predicted_label_ids = np.argmax(output.predictions, axis=1)
predicted_labels = [id2label[label_id] for label_id in predicted_label_ids.tolist()]

In [ ]:
out_path = model_dir / "predictions.jsonl"
with jsonlines.open(out_path, "w") as f:
    for label in predicted_labels:
        f.write({"label": label})

In [ ]:
!python xstance/evaluate.py --gold xstance/data/test.jsonl --pred {out_path.resolve()}

new_comments_defr
DE 76.34064932765187
FR 77.09480122324159

new_questions_defr
DE 72.91853784248686
FR 76.23040472108377

new_topics_defr
DE 72.513362814061
FR 74.34102696068727

new_comments_it
IT 71.63860405499884

